In [ ]:
import pandas as pd  
pd.options.plotting.backend = 'plotly'
pd.set_option('display.max_columns', None)

import numpy as np            

from scipy.stats import poisson        
from scipy.optimize import minimize    
from scipy.special import factorial

from numba import jit 

from sklearn.preprocessing import StandardScaler

import plotly
import plotly.express as px

In [ ]:
@jit()
def loglike(price, count, alfa, beta):
    '''
    Linear Log-Likelihood from demand exponential function:
    
    demand = np.exp(alfa*price + beta)
    
    args:
    price float : product price
    count int : product quantity sold in that price
    alfa float : first fit parameter, price parameter
    beta float : second fit parameter, intercept parameter
    '''
    
    demand = np.exp(alfa*price + beta)
    
    return -demand + count*np.log(demand) # - np.log(factorial(count))  

In [ ]:

def fit(datos, p1, p2,
        opciones={'disp':True,'maxiter':100},  # Funcion para obtener alfas, betas y gamma, stats y maximas iteraciones
        metodo='trust-constr',    # metodo Nelder-Mead, Powell, CG, BFGS, Newton-CG, L-BFGS-B, TNC, COBYLA, SLSQP, trust-constr, dogleg, trust-ncg, trust-exact, trust-krylov (tambien custom)
        ):  

    
    # inicializacion aleatoria de los parametros
    val_inicial=np.concatenate((np.array([p1]),   # alfa
                                np.array([p2])    # beta
                                ))                
  

    def menos_loglike(params):  # Funcion menos log-verosimilitud, funcion a minimizar

        log_like = [loglike(n.price,      # price from data
                            n.count_,     # count from data
                            params[0],    # alfa parameter
                            params[1])    # beta parameter
                    for n in datos.itertuples()] # seleccion en filas
        
        return -sum(log_like) 
        
    
    salida=minimize(menos_loglike, 
                    val_inicial, 
                    options=opciones, 
                    #constraints=restricciones, 
                    method=metodo) # minimizacion

    #return salida       # salida de la minimizacion
    return salida.x   # alfa, beta 
    

In [ ]:
%%time

df = pd.read_parquet('../data/final_data.parquet')

df.head()

In [ ]:
df.shape

In [ ]:
rates = [c for c in df.columns if '_rate' in c]

for r in rates:
    df[r]=df[r].apply(lambda x: x/100)

In [ ]:
df.head()

In [ ]:
df_lst = []


for r in rates:
    data = df.groupby(['date', r]).count().reset_index()
    
    data = data.iloc[:, 0:3]
    
    data.columns = ['date', 'price', 'count_']
    
    df_lst.append(data)

In [ ]:
params = fit(df_lst[0][['price', 'count_']], 
                 p1=-1., 
                 p2=0.,
                 opciones={'disp':True,'maxiter':10000})
    
y = np.exp(params[0]*x+params[1])*100



df_plot=pd.DataFrame()
df_plot['x']=x
df_plot['y']=y
df_plot['profit']=df_plot.x*df_plot.y


display(px.line(df_plot, x='x', y='y'))
px.line(df_plot, x='x', y='profit')

In [ ]:
x = np.linspace(1, 100, 100)

for d in df_lst:
    
    params = fit(d[['price', 'count_']], 
                 p1=-1., 
                 p2=0.,
                 opciones={'disp':True,'maxiter':10000})
    
    y = np.exp(params[0]*x+params[1])
    
    
    
    df_plot=pd.DataFrame()
    df_plot['x']=x
    df_plot['y']=y
    df_plot['profit']=df_plot.x*df_plot.y*100
    
    
    display(px.line(df_plot, x='x', y='y'))
    display(px.line(df_plot, x='x', y='profit'))

In [ ]:
df_lst[-1]

In [ ]:
params = fit(df_lst[-1][['price', 'count_']], 
                 p1=-1., 
                 p2=0.,
                 opciones={'disp':True,'maxiter':10000})
    
y = np.exp(params[0]*x+params[1])*100



df_plot=pd.DataFrame()
df_plot['x']=x
df_plot['y']=y
df_plot['profit']=df_plot.x*df_plot.y


display(px.line(df_plot, x='x', y='y'))
px.line(df_plot, x='x', y='profit')

In [ ]:
df_lst[-1].to_parquet('../data/fake_data3.parquet')